
## 2. Язык Julia: работа с таблицами данных

In [6]:
# Подключение пакетов
# Pkg.add("RDatasets")
using DataArrays, DataFrames, RDatasets

Функции для работы с таблицами данных реализованы в рассмотренном ранее пакете **DataFrames**, который работает поверх пакета **DataArrays**. Последний реализует особый тип данных `NA` для пропущенных значений (которого почему-то нет в языке Julia "из коробки", и это первая странность для языка с таким позиционированием), а также массивы `DataArray` - аналоги базовых массивов `Array`, но допускающие наличие пропущенных значений.

Наборы данных будем брать из пакета **RDatasets**, куда запакованы классические для изучающих R данные.

### 2.1. Пропущенные данные

Ниже представлены примеры из документации.

In [8]:
dv = @data([NA, 3, 2, 5, 4])

5-element DataArrays.DataArray{Int32,1}:
  NA
 3  
 2  
 5  
 4  

In [10]:
mean(dv)

NA

In [11]:
# dropna() - функция для удаления пропущенных значений.
mean(dropna(dv))

3.5

In [14]:
# Можно также заменить NA на какое-то число:
convert(Array, dv, 11)

5-element Array{Int32,1}:
 11
  3
  2
  5
  4

### 2.2. Создание таблиц данных; работа с факторами

Таблицы можно создавать как одной командой, так и последовательным добавлением столбцов.

In [16]:
df = DataFrame(A = 1:4, B = ["M", "F", "F", "M"])

,A,B
1,1,M
2,2,F
3,3,F
4,4,M


In [18]:
df = DataFrame()
df[:A] = 1:8
df[:B] = ["M", "F", "F", "M", "F", "M", "M", "F"]
df

,A,B
1,1,M
2,2,F
3,3,F
4,4,M
5,5,F
6,6,M
7,7,M
8,8,F


Чтобы узнать количество строк и столбцов, нужно использовать функцию `size()`.

In [21]:
size(df, 1), size(df, 2)

(8,2)

Начало (первые 2 строки) и конец (последние 4 строки) таблицы:

In [27]:
head(df, 2)

,A,B
1,1,M
2,2,F


In [31]:
tail(df, 4)

,A,B
1,5,F
2,6,M
3,7,M
4,8,F


По умолчанию в роли факторов выступают обычные столбцы с данными текстового типа. Для создания "настоящих" факторов (как в R) нужно применить к столбцу функцию `pool()`. Это обеспечит более компактное представление данных в памяти, а также позволит корректно задавать спецификации моделей (будет рассматриваться далее).

In [30]:
dv = @data(["Group A", "Group A", "Group A",
            "Group B", "Group B", "Group B"])
pdv = pool(dv)
levels(pdv)

2-element Array{ASCIIString,1}:
 "Group A"
 "Group B"

Также можно модицифировать таблицу без операции присвоения (изменение "на месте" - in-place), причем несколько столбцов за раз:

In [33]:
df = DataFrame(A = [1, 1, 1, 2, 2, 2],
               B = ["X", "X", "X", "Y", "Y", "Y"])
pool!(df, [:A, :B])

,A,B
1,1,X
2,1,X
3,1,X
4,2,Y
5,2,Y
6,2,Y


### 2.3. Индексирование таблиц; создание поднаборов


In [36]:
df = DataFrame(A = 1:10, B = 2:2:20)
# Выбор строк:
df[1:3, :]

,A,B
1,1,2
2,2,4
3,3,6


In [38]:
# Выбор столбцов по индексу или по имени:
df[1]
df[:A]

10-element DataArrays.DataArray{Int32,1}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10

In [39]:
# Выбор отдельных наблюдений:
df[1, 1]
df[1, :A]

1

In [40]:
# Выбор поднаборов (порядок столбцов имеет значение):
df[1:3, [:A, :B]]

,A,B
1,1,2
2,2,4
3,3,6


In [44]:
# Выбор по условию:
df[df[:A] % 2 .== 0, :]

,A,B
1,2,4
2,4,8
3,6,12
4,8,16
5,10,20


### 2.4. Сортировка (упорядочивание)
Рассмотрим комплексный пример из документации. Набор данных `iris` будет отсортирован сперва по столбцу `Species` в лексикографическом порядке (значения предварительно приводятся к верхнему регистру - функция `uppercase`), а затем - по убыванию значений столбца `SepalLength`.

In [45]:
iris = dataset("datasets", "iris")
sort!(iris, cols = [order(:Species, by = uppercase),
                    order(:SepalLength, rev = true)])

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
1,5.8,4.0,1.2,0.2,setosa
2,5.7,4.4,1.5,0.4,setosa
3,5.7,3.8,1.7,0.3,setosa
4,5.5,4.2,1.4,0.2,setosa
5,5.5,3.5,1.3,0.2,setosa
6,5.4,3.9,1.7,0.4,setosa
7,5.4,3.7,1.5,0.2,setosa
8,5.4,3.9,1.3,0.4,setosa
9,5.4,3.4,1.7,0.2,setosa
10,5.4,3.4,1.5,0.4,setosa


Поддерживается синтаксис следующего вида:

In [50]:
sort!(iris, cols = (:Species, :SepalLength, :SepalWidth),
            rev = (true, false, false))
# Аналог:
sort!(iris,
      cols = (order(:Species, rev = true), :SepalLength, :SepalWidth))

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
1,4.9,2.5,4.5,1.7,virginica
2,5.6,2.8,4.9,2.0,virginica
3,5.7,2.5,5.0,2.0,virginica
4,5.8,2.7,5.1,1.9,virginica
5,5.8,2.7,5.1,1.9,virginica
6,5.8,2.8,5.1,2.4,virginica
7,5.9,3.0,5.1,1.8,virginica
8,6.0,2.2,5.0,1.5,virginica
9,6.0,3.0,4.8,1.8,virginica
10,6.1,2.6,5.6,1.4,virginica


### 2.5. "Широкий" и "длинный" формат данных

Форматирование данных из "широкого" в "длинный" формат осуществляется по аналогии с пакетом **reshape2** для R.

"Длинный" формат (значения всех переменных в одном столбце):

In [87]:
iris = dataset("datasets", "iris")
iris[:id] = 1:size(iris, 1)  
d = stack(iris, [:SepalLength, :SepalWidth, :PetalLength, :PetalWidth])
# Аналогично: d = stack(iris, [1:4])

,variable,value,Species,id
1,SepalLength,5.1,setosa,1
2,SepalLength,4.9,setosa,2
3,SepalLength,4.7,setosa,3
4,SepalLength,4.6,setosa,4
5,SepalLength,5.0,setosa,5
6,SepalLength,5.4,setosa,6
7,SepalLength,4.6,setosa,7
8,SepalLength,5.0,setosa,8
9,SepalLength,4.4,setosa,9
10,SepalLength,4.9,setosa,10


"Широкий" формат:

In [96]:
widedf = unstack(d, :variable, :value)

,Species,id,PetalLength,PetalWidth,SepalLength,SepalWidth
1,setosa,1,1.4,0.2,5.1,3.5
2,setosa,5,1.4,0.2,4.9,3.0
3,setosa,9,1.3,0.2,4.7,3.2
4,setosa,13,1.5,0.2,4.6,3.1
5,setosa,17,1.4,0.2,5.0,3.6
6,setosa,21,1.7,0.4,5.4,3.9
7,setosa,25,1.4,0.3,4.6,3.4
8,setosa,29,1.5,0.2,5.0,3.4
9,setosa,33,1.4,0.2,4.4,2.9
10,setosa,37,1.5,0.1,4.9,3.1


### 2.6. Итоговые статистики; стратегия "Split-Apply-Combine"
Прежде всего нужно отметить возможность вычисления итоговых статистик для всех (или нескольких) столбцов таблицы данных:

In [129]:
describe(iris[1:3])

SepalLength
Min      4.3
1st Qu.  5.1
Median   5.8
Mean     5.843333333333332
3rd Qu.  6.4
Max      7.9
NAs      0
NA%      0.0%

SepalWidth
Min      2.0
1st Qu.  2.8
Median   3.0
Mean     3.0573333333333337
3rd Qu.  3.3
Max      4.4
NAs      0
NA%      0.0%

PetalLength
Min      1.0
1st Qu.  1.6
Median   4.35
Mean     3.7580000000000005
3rd Qu.  5.1
Max      6.9
NAs      0
NA%      0.0%



Кроме того, можно применять те или иный функции к совокупности столбцов при помощи функции `colwise()` (аналог `apply()` в R): 

In [117]:
df = DataFrame(a = rep(1:4, 2), b = rep(2:-1:1, 4), c = randn(8))
colwise(sum, df)

3-element Array{Any,1}:
 [20]               
 [12]               
 [7.510390532304938]

Реализована стратегия "Split-Apply-Combine", описанная в [The Split-Apply-Combine Strategy for Data Analysis](https://www.jstatsoft.org/article/view/v040i01):

In [121]:
df = DataFrame(a = rep(1:4, 2), b = rep(2:-1:1, 4), c = randn(8))
# Вычисляем сумму значений столбца "c" с группировкой по столбцу "a":
by(df, :a, d -> sum(d[:c]))

,a,x1
1,1,2.251006808623205
2,2,1.4860318319904267
3,3,-1.060231557584403
4,4,3.5259613652270683


In [122]:
# Группировка по столбцу "Species", для каждой группы считаем среднее и дисперсию "PetalLength":
by(iris, :Species) do df
    DataFrame(m = mean(df[:PetalLength]), s² = var(df[:PetalLength]))
end

,Species,m,s²
1,setosa,1.462,0.030159183673469384
2,versicolor,4.26,0.22081632653061226
3,virginica,5.552,0.3045877551020408


In [123]:
# Средние и медианы для всех переменных с группировкой по столбцу "Species":
aggregate(iris, :Species, [sum, mean])

,Species,SepalLength_sum,SepalLength_mean,SepalWidth_sum,SepalWidth_mean,PetalLength_sum,PetalLength_mean,PetalWidth_sum,PetalWidth_mean,id_sum,id_mean
1,setosa,250.3,5.006,171.39999999999998,3.4279999999999995,73.1,1.462,12.300000000000002,0.24600000000000005,1275,25.5
2,versicolor,296.79999999999995,5.935999999999999,138.5,2.77,213.0,4.26,66.3,1.3259999999999998,3775,75.5
3,virginica,329.40000000000003,6.588000000000001,148.7,2.9739999999999998,277.59999999999997,5.552,101.29999999999998,2.026,6275,125.5


In [124]:
# Размер каждой из подгрупп, заданных по столбцу "Species":
for subdf in groupby(iris, :Species)
    println(size(subdf, 1))
end

50
50
50


### 2.7. Объединения таблиц
Таблицы данных можно объединять по тем же принципам, что и таблицы SQL.

In [132]:
a = DataFrame(ID = [1, 2], Name = ["A", "B"])

,ID,Name
1,1,A
2,2,B


In [133]:
b = DataFrame(ID = [1, 3], Job = ["Doctor", "Lawyer"])

,ID,Job
1,1,Doctor
2,3,Lawyer


In [134]:
join(a, b, on = :ID, kind = :inner)

,ID,Name,Job
1,1,A,Doctor


In [135]:
join(a, b, on = :ID, kind = :left)

,ID,Name,Job
1,1,A,Doctor
2,2,B,NA


In [136]:
join(a, b, on = :ID, kind = :right)

,Name,ID,Job
1,A,1,Doctor
2,NA,3,Lawyer


In [137]:
join(a, b, on = :ID, kind = :outer)

,ID,Name,Job
1,1,A,Doctor
2,2,B,NA
3,3,NA,Lawyer


In [138]:
join(a, b, on = :ID, kind = :semi)

,ID,Name
1,1,A


In [139]:
join(a, b, on = :ID, kind = :anti)

,ID,Name
1,2,B


In [140]:
# Переименование столбцов:
# rename!(df3, {:old_name => :new_name, :another_old_name => :another_new_name})

### Выводы
В пакете **DataFrames**, на первый взгляд, есть все необходимое для работы с таблицами данных. Воспроизведен функционал пакетов **dplyr** и **reshape2** для R, местами прослеживается сходство с подходами, использованными в **data.table** (например, изменение таблиц "на месте"). Синтаксис несложный и привычный для пользователей вышеназванных пакетов. 